In [5]:
import pandas as pd
import panel as pn
from vega import Vega
pn.extension('vega')

In [6]:
from bokeh.plotting import figure, output_notebook, show
output_notebook()

Loading BokehJS ...

In [236]:
range(0, len(list_of_links_to_add))

range(0, 24753)

In [20]:
def plot_history_x(hist_x):
    p = figure(title="History of nr of links tested",plot_width=600, plot_height=300)
    p.line(range(0, len(hist_x)), hist_x)
    return show(p)
# plot_distances_to_add(list_of_links_to_add)

In [19]:
def plot_distances_to_add(list_of_links_to_add):
    ys = list(map(lambda x:x['highDd'], list_of_links_to_add))
    p = figure(title="Sorted list of links to add",plot_width=600, plot_height=300)
    p.xaxis.axis_label = "Order in list"
    p.yaxis.axis_label = "Distance within this link"
    p.line(range(0, len(list_of_links_to_add)), ys)
    return show(p)
# plot_distances_to_add(list_of_links_to_add)

In [18]:
def plot_history_xy(hist_x, hist_y):
    xs = []
    ys = []
    for i in range(0, len(hist_x)):
        xs.append(hist_x[i][0])
        ys.append(hist_y[i])

    TOOLTIPS = [
        ("index", "$index"),
        ("(x,y)", "($x, $y)")
    ]

    p = figure(plot_width=400, plot_height=400,tooltips=TOOLTIPS)

    sorted_ys = [y for _,y in sorted(zip(xs,ys))]
    p.line(sorted(xs), sorted_ys, line_width=2)

    return show(p)
# plot_history_xy(hist_x, hist_y)

In [17]:
def plot_matrix(matrix):
    size = len(matrix)
    p = figure(x_range=(0, size), y_range=(0, size), tooltips=[("index", "$index"),("x", "$x"), ("y", "$y")])
    p.image(image=[matrix], x=0, y=0, dw=size, dh=size, palette="Blues256")
    return show(p)
# plot_matrix(dist_matrix)

# Load STAD

In [8]:
%load_ext autoreload
%autoreload 2
import __init__ as stad

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Run stuff

In [9]:
data = pd.read_csv('data/five_circles.csv', header=0)#.sample(n=500)
values = data[['x','y']].values.tolist()
xs = data['x'].values.tolist()
ys = data['y'].values.tolist()
# zs = data['z'].values.tolist()
highD_dist_matrix = stad.calculate_highD_dist_matrix(values)
[g,hist_x,hist_y] = stad.run_stad(highD_dist_matrix,features={'x':xs, 'y':ys}, debug=False)
# plot_history(g,hist_x, hist_y)

In [10]:
dist_matrix = highD_dist_matrix

In [11]:
mst = stad.create_mst(dist_matrix, [], {'x':xs, 'y':ys})

In [13]:
?stad.create_mst

In [14]:
all_links = stad.create_list_of_all_links_with_values(dist_matrix)

In [15]:
list_of_links_to_add = stad.create_list_of_links_to_add(all_links, mst)

In [21]:
plot_distances_to_add(list_of_links_to_add)

In [278]:
class MyTakeStep(object):
    def __init__(self, stepsize=1000, min_links=0, max_links=99999):
        self.stepsize = stepsize
        self.min_links = min_links
        self.max_links = max_links
    def __call__(self, x):
        s = self.stepsize
        x = int(np.random.normal(x, s))
        if ( x < self.min_links ):
            x = self.min_links
        if ( x > self.max_links ):
            x = self.max_links
        return x

In [259]:
history_x = []
history_y = []
def cost_function(nr_of_links, args):
    graph = args['graph']
    list_of_links_to_add = args['list_of_links_to_add']
    highD_dist_matrix = args['highD_dist_matrix']

    new_graph = stad.add_links_to_graph(graph, highD_dist_matrix, list_of_links_to_add, nr_of_links)
    correlation = stad.correlation_between_distance_matrices(highD_dist_matrix, list(stad.graph_to_distancematrix(new_graph)))

    history_x.append(nr_of_links)
    history_y.append(correlation)

    return correlation

In [22]:
minimizer_kwargs = {'args':{'graph':mst,
                            'list_of_links_to_add':list_of_links_to_add,
                            'highD_dist_matrix':highD_dist_matrix}}
result = stad.optimize.basinhopping(
    cost_function,
    len(mst.es()),
    disp=False,
    T=0.5, interval=100,
    minimizer_kwargs=minimizer_kwargs,
    take_step=MyTakeStep(stepsize=100, min_links=len(mst.es()), max_links=9999)
)

NameError: name 'cost_function' is not defined

In [281]:
result

                        fun: 0.9690090930637903
 lowest_optimization_result:       fun: 0.9690090930637903
 hess_inv: array([[1]])
      jac: array([0.])
  message: 'Optimization terminated successfully.'
     nfev: 3
      nit: 0
     njev: 1
   status: 0
  success: True
        x: array([401.])
                    message: ['requested number of basinhopping iterations completed successfully']
      minimization_failures: 0
                       nfev: 303
                        nit: 100
                       njev: 101
                          x: array([401.])

In [282]:
plot_history(history_x, history_y)

In [284]:
plot_history_x(history_x)

In [166]:
def plot_history(g, hist_x, hist_y):
    history_data = []
    max_y = 0
    min_y = 1
    for i in range(0, len(hist_x)):
        if hist_y[i] > max_y: max_y = hist_y[i]
        if hist_y[i] < min_y: min_y = hist_y[i]
        history_data.append({'x': hist_x[i][0], 'y': hist_y[i]})

    # history_data = str(history_data).replace("'", '"')
#     print(history_data)

    x = len(g.es())
    history_spec = {
        "$schema": "https://vega.github.io/schema/vega/v5.json",
        "width": 400,
        "height": 200,
        "padding": 5,

        "data": [
            {
                "name": "table",
                "values": history_data,
                "transform": [
                  {
                    "type": "collect",
                    "sort": {"field": "x"}
                  }
                ]
            },
            {
                "name": "rule-data",
                "values": [{"x": x}]
            }
        ],

        "scales": [
            {
                "name": "xscale",
                "domain": {"data": "table", "field": "x"},
                "range": "width"
            },
            {
                "name": "yscale",
                "domain": [min_y, max_y],
                "range": "height"
            }
        ],
        "axes": [
          {"orient": "bottom", "scale": "xscale"},
          {"orient": "left", "scale": "yscale"}
        ],
        "marks": [
            {
                "type": "line",
                "from": {"data":"table"},
                "encode": {
                    "enter": {
                        "x": {"scale": "xscale", "field": "x", "sort": "ascending"},
                        "y": {"scale": "yscale", "field": "y"}
                    }
                }
            },
            {
                "type": "symbol",
                "from": {"data": "rule-data"},
                "encode": {
                    "enter": {
                        "x": {"field": "x", "scale": "xscale"},
                        "fill": {"value": "red"},
                        "stroke": {"value": "red"}
                    }
                }
            }
        ]
    }
    return pn.pane.Vega(history_spec)

In [107]:
stad.draw_stad(g)

In [50]:
def create_vega_nodes(graph):
    output = []
    # The following will automatically pick up the lens...
    feature_labels = graph.vs()[0].attributes().keys()
    for v in graph.vs():
        node = {"name": v.index}
        for f in feature_labels:
            node[f] = v.attributes()[f]
        output.append(node)
    return output

def create_vega_links(graph):
    output = []
    for e in graph.es():
        output.append({"source": e.source, "target": e.target, "value": e.attributes()['distance']})
    return output

In [53]:
# print('{"nodes":' + str(nodes).replace("'", '"').replace('},',"},\n") + ', "links":' + str(links).replace("'", '"').replace('},',"},\n") + '}')

In [75]:
nodes = create_vega_nodes(g)
links = create_vega_links(g)
spec = {
  "$schema": "https://vega.github.io/schema/vega/v5.json",
  "width": 1200,
  "height": 600,
  "padding": 0,
  "autosize": "none",

  "signals": [
    { "name": "cx", "update": "width / 4" },
    { "name": "cy", "update": "height / 2" },
    {
      "description": "State variable for active node dragged status.",
      "name": "dragged", "value": 0,
      "on": [
        {
          "events": "symbol:mouseout[!event.buttons], window:mouseup",
          "update": "0"
        },
        {
          "events": "symbol:mouseover",
          "update": "dragged || 1"
        },
        {
          "events": "[symbol:mousedown, window:mouseup] > window:mousemove!",
          "update": "2", "force": True
        }
      ]
    },
    {
      "description": "Graph node most recently interacted with.",
      "name": "dragged_node", "value": None,
      "on": [
        {
          "events": "symbol:mouseover",
          "update": "dragged === 1 ? item() : dragged_node"
        }
      ]
    },
    {
      "description": "Flag to restart Force simulation upon data changes.",
      "name": "restart", "value": False,
      "on": [
        {"events": {"signal": "dragged"}, "update": "dragged > 1"}
      ]
    },
    { "description": "Any datapoint is activated",
      "name": "datapoint_is_activated", "value": False,
      "on": [
        {
          "events": "symbol:mouseover",
          "update": "true"
        },
        {
          "events": "symbol:mouseout",
          "update": "false"
        }
      ]
    },
    { "description": "Active datapoint",
      "name": "activated_datapoint", "value": None,
      "on": [
        {
          "events": "symbol:mouseover",
          "update": "item()"
        },
        {
          "events": "symbol:mouseout",
          "update": "null"
        }
      ]
    }
  ],

  "data": [
    {
      "name": "node-data",
      "values": nodes
    },
    {
      "name": "link-data",
      "values": links
    }
  ],
  "layout": {"padding": 20},
  "marks": [
    {
      "type": "group",
      "encode": {
        "update": {
          "width": {"value": 550},
          "height": {"value": 550}
        }
      },
      "marks": [
        {
          "name": "nodes",
          "type": "symbol",
          "zindex": 1,

          "from": {"data": "node-data"},
          "on": [
            {
              "trigger": "dragged",
              "modify": "dragged_node",
              "values": "dragged === 1 ? {fx:dragged_node.x, fy:dragged_node.y} : {fx:x(), fy:y()}"
            },
            {
              "trigger": "!dragged",
              "modify": "dragged_node", "values": "{fx: null, fy: null}"
            }
          ],

          "encode": {
            "enter": {
              "opacity": {"value": 0.5}
            },
            "update": {
              "size": [
                {"test": "datapoint_is_activated && datum === activated_datapoint.datum",
                "value": 300},
                {"value": 50}
              ],
              "cursor": {"value": "pointer"},
              "fill": [
                {"test": "datapoint_is_activated && datum === activated_datapoint.datum",
                "value": "red"},
                {"value": "steelblue"}
              ]
            }
          },

          "transform": [
            {
              "type": "force",
              "iterations": 300,
              "velocityDecay": 0.4,
              "restart": {"signal": "restart"},
              "static": False,
              "forces": [
                {"force": "center", "x": {"signal": "cx"}, "y": {"signal": "cy"}},
                {"force": "collide", "radius": 2},
                {"force": "nbody", "strength": -0.5},
                {"force": "link", "links": "link-data", "distance": 5}
              ]
            }
          ]
        },
        {
          "type": "path",
          "from": {"data": "link-data"},
          "interactive": False,
          "encode": {
            "update": {
              "stroke": {"value": "lightgrey"}
            }
          },
          "transform": [
            {
              "type": "linkpath", "shape": "line",
              "sourceX": "datum.source.x", "sourceY": "datum.source.y",
              "targetX": "datum.target.x", "targetY": "datum.target.y"
            }
          ]
        }
      ]
    },
    {
      "type": "group",
      "style": "cell",
      "encode": {
        "update": {
          "width":  {"value": 600},
          "height": {"value": 600}
        }
      },
      "scales": [
        {
          "name": "x",
          "type": "linear",
          "round": True,
          "nice": True,
          "zero": True,
          "domain": {"data": "node-data", "field": "y"},
          "range": [580,20]
        },
        {
          "name": "y",
          "type": "linear",
          "round": True,
          "nice": True,
          "zero": True,
          "domain": {"data": "node-data", "field": "z"},
          "range": [20,580]
        }
      ],
      "marks": [
        {
          "type": "symbol",
          "from": {"data": "node-data"},
          "encode": {
            "update": {
              "y": {"scale": "x", "field": "y"},
              "x": {"scale": "y", "field": "z"},
              "shape": {"value": "circle"},
              "opacity": {"value": 0.5},
              "fill": [
                {"test": "datapoint_is_activated && datum === activated_datapoint.datum",
                "value": "red"},
                {"value": "steelblue"}
              ],
              "size": [
                {"test": "datapoint_is_activated && datum === activated_datapoint.datum",
                "value": 300},
                {"value": 50}
              ]
            }
          }
        }
      ]
    }
  ]
}
# pn.pane.Vega(spec).show()

In [69]:
history_data

'{"values": [{"x": 499.0, "y": 0.2571856948798701}, {"x": 499.00000001490116, "y": 0.2571856948798701}, {"x": 499.0, "y": 0.2571856948798701}, {"x": 499.2589378484326, "y": 0.2571856948798701}, {"x": 499.25893786333376, "y": 0.2571856948798701}, {"x": 499.2589378484326, "y": 0.2571856948798701}, {"x": 499.0377931617418, "y": 0.2571856948798701}, {"x": 499.03779317664294, "y": 0.2571856948798701}, {"x": 499.0377931617418, "y": 0.2571856948798701}, {"x": 499.14164146247504, "y": 0.2571856948798701}, {"x": 499.1416414773762, "y": 0.2571856948798701}, {"x": 499.14164146247504, "y": 0.2571856948798701}, {"x": 499.11468917098057, "y": 0.2571856948798701}, {"x": 499.11468918588173, "y": 0.2571856948798701}, {"x": 499.11468917098057, "y": 0.2571856948798701}, {"x": 498.68225335862337, "y": 0.2571858260125698}, {"x": 498.68225337352453, "y": 0.2571858260125698}, {"x": 498.68225335862337, "y": 0.2571858260125698}, {"x": 498.6183675512546, "y": 0.2571858260125698}, {"x": 498.61836756615577, "y": 

In [ ]:
{
  "$schema": "https://vega.github.io/schema/vega/v5.json",
  "width": 1200,
  "height": 600,
  "padding": 0,
  "autosize": "none",

  "signals": [
    { "name": "cx", "update": "width / 4" },
    { "name": "cy", "update": "height / 2" },
    {
      "description": "State variable for active node dragged status.",
      "name": "dragged", "value": 0,
      "on": [
        {
          "events": "symbol:mouseout[!event.buttons], window:mouseup",
          "update": "0"
        },
        {
          "events": "symbol:mouseover",
          "update": "dragged || 1"
        },
        {
          "events": "[symbol:mousedown, window:mouseup] > window:mousemove!",
          "update": "2", "force": true
        }
      ]
    },
    {
      "description": "Graph node most recently interacted with.",
      "name": "dragged_node", "value": null,
      "on": [
        {
          "events": "symbol:mouseover",
          "update": "dragged === 1 ? item() : dragged_node"
        }
      ]
    },
    {
      "description": "Flag to restart Force simulation upon data changes.",
      "name": "restart", "value": false,
      "on": [
        {"events": {"signal": "dragged"}, "update": "dragged > 1"}
      ]
    },
    { "description": "Any datapoint is activated",
      "name": "datapoint_is_activated", "value": false,
      "on": [
        {
          "events": "symbol:mouseover",
          "update": "true"
        },
        {
          "events": "symbol:mouseout",
          "update": "false"
        }
      ]
    },
    { "description": "Active datapoint",
      "name": "activated_datapoint", "value": null,
      "on": [
        {
          "events": "symbol:mouseover",
          "update": "item()"
        },
        {
          "events": "symbol:mouseout",
          "update": "null"
        }
      ]
    }
  ],

  "data": [
    {
      "name": "node-data",
      "url": "https://gist.githubusercontent.com/jandot/e53e1b9c6a10f6b7ef3125136ac38ea3/raw/10b6ba02732cdff1f585dd2568671ec56ef9b67c/horse_network.json",
      "format": {"type": "json", "property": "nodes"}
    },
    {
      "name": "link-data",
      "url": "https://gist.githubusercontent.com/jandot/e53e1b9c6a10f6b7ef3125136ac38ea3/raw/10b6ba02732cdff1f585dd2568671ec56ef9b67c/horse_network.json",
      "format": {"type": "json", "property": "links"}
    }
  ],
  "layout": {"padding": 20},
  "marks": [
    {
      "type": "group",
      "encode": {
        "update": {
          "width": {"value": 550},
          "height": {"value": 550}
        }
      },
      "marks": [
        {
          "name": "nodes",
          "type": "symbol",
          "zindex": 1,

          "from": {"data": "node-data"},
          "on": [
            {
              "trigger": "dragged",
              "modify": "dragged_node",
              "values": "dragged === 1 ? {fx:dragged_node.x, fy:dragged_node.y} : {fx:x(), fy:y()}"
            },
            {
              "trigger": "!dragged",
              "modify": "dragged_node", "values": "{fx: null, fy: null}"
            }
          ],

          "encode": {
            "enter": {
              "opacity": {"value": 0.5}
            },
            "update": {
              "size": [
                {"test": "datapoint_is_activated && datum === activated_datapoint.datum",
                "value": 300},
                {"value": 50}
              ],
              "cursor": {"value": "pointer"},
              "fill": [
                {"test": "datapoint_is_activated && datum === activated_datapoint.datum",
                "value": "red"},
                {"value": "steelblue"}
              ]
            }
          },

          "transform": [
            {
              "type": "force",
              "iterations": 300,
              "velocityDecay": 0.4,
              "restart": {"signal": "restart"},
              "static": false,
              "forces": [
                {"force": "center", "x": {"signal": "cx"}, "y": {"signal": "cy"}},
                {"force": "collide", "radius": 2},
                {"force": "nbody", "strength": -0.5},
                {"force": "link", "links": "link-data", "distance": 5}
              ]
            }
          ]
        },
        {
          "type": "path",
          "from": {"data": "link-data"},
          "interactive": false,
          "encode": {
            "update": {
              "stroke": {"value": "lightgrey"}
            }
          },
          "transform": [
            {
              "type": "linkpath", "shape": "line",
              "sourceX": "datum.source.x", "sourceY": "datum.source.y",
              "targetX": "datum.target.x", "targetY": "datum.target.y"
            }
          ]
        }
      ]
    },
    {
      "type": "group",
      "style": "cell",
      "encode": {
        "update": {
          "width":  {"value": 600},
          "height": {"value": 600}
        }
      },
      "scales": [
        {
          "name": "x",
          "type": "linear",
          "round": true,
          "nice": true,
          "zero": true,
          "domain": {"data": "node-data", "field": "y"},
          "range": [580,20]
        },
        {
          "name": "y",
          "type": "linear",
          "round": true,
          "nice": true,
          "zero": true,
          "domain": {"data": "node-data", "field": "z"},
          "range": [20,580]
        }
      ],
      "marks": [
        {
          "type": "symbol",
          "from": {"data": "node-data"},
          "encode": {
            "update": {
              "y": {"scale": "x", "field": "y"},
              "x": {"scale": "y", "field": "z"},
              "shape": {"value": "circle"},
              "opacity": {"value": 0.5},
              "fill": [
                {"test": "datapoint_is_activated && datum === activated_datapoint.datum",
                "value": "red"},
                {"value": "steelblue"}
              ],
              "size": [
                {"test": "datapoint_is_activated && datum === activated_datapoint.datum",
                "value": 300},
                {"value": 50}
              ]
            }
          }
        }
      ]
    }
  ]
}
